<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/CNN_Inception_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [3]:
train_data = datasets.MNIST(root = "MNIST/processed/training.pt", train = True, transform=transforms.ToTensor(), download= True)
test_data = datasets.MNIST(root = "MNIST/processed/test.pt", train = False, transform=transforms.ToTensor(), download = True)

Extracting MNIST/processed/training.pt/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/processed/training.pt/MNIST/raw


Extracting MNIST/processed/training.pt/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/processed/training.pt/MNIST/raw


Extracting MNIST/processed/training.pt/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/processed/training.pt/MNIST/raw


Extracting MNIST/processed/training.pt/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/processed/training.pt/MNIST/raw
Processing...
Done!





/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Extracting MNIST/processed/test.pt/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/processed/test.pt/MNIST/raw


Extracting MNIST/processed/test.pt/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/processed/test.pt/MNIST/raw


Extracting MNIST/processed/test.pt/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/processed/test.pt/MNIST/raw





Extracting MNIST/processed/test.pt/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/processed/test.pt/MNIST/raw
Processing...
Done!


In [0]:
train_loader = DataLoader(dataset = train_data, batch_size= 64, shuffle= True)
test_loader = DataLoader(dataset = test_data, batch_size= 64, shuffle= False)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is ", device)

device is  cpu


In [0]:
class inception(torch.nn.Module):
  def __init__(self, input_channel):
    super(inception, self).__init__()

    self.in_channel = input_channel

    self.b1x1 = torch.nn.Conv2d(in_channels= self.in_channel, out_channels=16, kernel_size=1 )

    self.b241x1 = torch.nn.Conv2d(in_channels=self.in_channel, out_channels= 24, kernel_size=1 )

    self.b5x5_1 = torch.nn.Conv2d(in_channels=self.in_channel, out_channels=16, kernel_size=1 )
    self.b5x5_2 = torch.nn.Conv2d(in_channels=16, out_channels=24, kernel_size=5, padding =2 )

    self.b3x3_1 = torch.nn.Conv2d(in_channels=self.in_channel, out_channels=16, kernel_size=1 )
    self.b3x3_2 = torch.nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3 )
    self.b3x3_3 = torch.nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3 )
    
    def forward(self, image):
      in_size = image.size(0)# it will give input shape

      branch1x1 = self.b1x1(image)

      branch_pool = F.avg_pool2d(image, kernel_size=3, stride=1, padding=1)
      branch241x1 = self.b241x1(branch_pool)

      branch5x5 = self.b5x5_2(self.b5x5_1(image))

      branch3x3 = self.b3x3_3(self.b3x3_2(self.b3x3_1(image)))

      output = [branch1x1, branch5x5, branch3x3, branch241x1] #what is sape of each

      return torch.cat(output, 1)

In [0]:
class mnist(torch.nn.Module):
  def __init__(self):
    super(mnist,self).__init__()
    
    self.incepA= inception(input_channel=10)
    self.linear = torch.nn.Linear(20,10)



  def forward(self, image):
    in_size = image.size(0)
    outputA = self.incepA(image)
    outputA = self.linear(outputA)
    outputA = F.relu(outputA)
    outputA = outputA.view(in_size, -1)
    outputA = self.linear(outputA)
    return F.log_softmax(x)


In [0]:
model = mnist()

In [0]:
# class mnist(torch.nn.Module):
#   def __init__(self):
#     super(mnist, self).__init__()
#     self.conv1 = torch.nn.Conv2d(1, 10 , kernel_size= 5, stride= 1, padding= 0)
#     self.conv2 = torch.nn.Conv2d(10, 20, kernel_size= 5, stride= 1, padding= 0)
#     self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
#     self.linear = torch.nn.Linear(320,10) #how 320 comes up

#   def forward(self, data):
#     in_size = data.size(0)
#     x = F.relu(self.maxpool(self.conv1(data)))
#     x = F.relu(self.maxpool(self.conv2(x)))  
#     x = x.view(in_size, -1) 
#     x = self.linear(x)
#     out = F.log_softmax(x)
#     return out

needs to undedrstand mistake below

In [0]:
# class mnist(torch.nn.Module):
#   def __init__(self):
#     super(mnist, self).__init__()
#     self.conv1 = torch.nn.Conv2d(1, 10 , kernel_size= 5, stride= 1, padding= 0)
#     self.conv2 = torch.nn.Conv2d(10, 20, kernel_size= 5, stride= 1, padding= 0) ()
#     self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
#     self.linear = torch.nn.Linear(320,10) #how 320 comes up

#   def forward(self, data):
#     in_size = data.size(0)
#     x = torch.nn.ReLU(self.maxpool(self.conv1(data))) # this relu is not givenng tensort output
#     x = torch.nn.ReLU(self.maxpool(self.conv2(x)))  
#     x = torch.view(in_size, -1)  # x.view
#     x = self.linear(x)
#     out = torch.nn.functional.log_softmax(x)
#     return out

In [22]:
model = mnist()
model.to(device)

mnist(
  (incepA): inception(
    (b1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (b241x1): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
    (b5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (b5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (b3x3_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (b3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1))
    (b3x3_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1))
  )
  (linear): Linear(in_features=20, out_features=10, bias=True)
)

Loss

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01, momentum=.4)

trainig

In [0]:
# def train(epoch):
#   model.train()
#   for Epo in range(epoch):
#     for batch_idx, (images, labels) in enumerate(train_loader):
#       images, labels = images.to(device), labels.to(device)
#       targets = model(images)
#       optimizer.zero_grad()
#       loss = criterian(targets, labels)
#       loss.backward()
#       optimizer.step()
#       if batch_idx % 500  == 0:
#         # print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}% | Loss: {:.6f}', format(
#         #     epoch, batch_idx*len(data), len(train_loader.dataset), 100. *batch_idx / len(train_loader), loss.item()
#         # ))
#         print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))



In [0]:

def train(epoch):
  model.train()
  for Epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      logit = model(data)
      loss = criterion(logit, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 500  == 0:
        # print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}% | Loss: {:.6f}', format(
        #     epoch, batch_idx*len(data), len(train_loader.dataset), 100. *batch_idx / len(train_loader), loss.item()
        # ))
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test():
  model.eval()  # what is meaning of it ?
  with torch.no_grad(): # its not compulsary but it will save some merory
    test_loss =0
    correct = 0
    for data, target in test_loader: # enumeration is not required as this is a single batch
      data, target = data.to(device), target.to(device)
      output = model(data)  # output.shape = 64,10
      #sum up the match loss
      test_loss += criterion(output, target).item()
      #get the index of the max value
      pred = output.data.max(1, keepdim=True)[1]  #output.data is ouput data/value ? what is max function https://www.journaldev.com/39463/pytorch-torch-max
      #1 is axis: 0 is row (ie colum wise max:> 1, 10, and 1 is colum ie row wise max)
      # provides max data and and its index
      correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
            f'({100. * correct / len(test_loader.dataset):.0f}%)')
    # print(f'====================\n test set : Average loss: {test_loss: .4f}, Accuracy: {correct}/{len(test_loader.dataset)} ' 
            # f'({100. * correct/len(test_loader.dataset):. 0f}%)')
            
  #         ===========================
  # Test set: Average loss: 0.0016, Accuracy: 9787/10000 (98%)
  # Testing timr: 0m 25s

In [27]:
if __name__ == '__main__':
  since = time.time()
  for epoch in range(1,2):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training timr: {m:.0f}m {s:.0f}s')
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing timr: {m:.0f}m {s:.0f}s')
  m, s = divmod(time.time() - since, 60)
  print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

NotImplementedError: ignored